## 0.1 处理绘图中会出现的异常

In [18]:
import os

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
# -*- coding: utf-8 -*-
"""
Created on Thu Sep 15 14:18:27 2022
@author: Kiramei
"""

'\nCreated on Thu Sep 15 14:18:27 2022\n@author: Kiramei\n'

## 1.1 导入相关数据以及库文件
1、 库文件包括numpy、pandas、matplotlib、torch等
2、 数据包括训练数据和测试数据

In [19]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from matplotlib import pyplot as plt

In [20]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# 展示数据形状
print('训练数据大小：', train_data.shape)
print('测试数据大小：', test_data.shape)

# 展示部分特征
print('部分训练特征：')
print(train_data.iloc[0:4, [0, 1, 2, 3, -3, -2, -1]])

# 数据构型 
print('训练数据构型：')
train_data.head()  ## getting first five rows
train_data.tail()  ## getting last five rows
train_data.info()  ## getting the information from the dataframe

训练数据大小： (1460, 81)
测试数据大小： (1459, 80)
部分训练特征：
   Id  MSSubClass MSZoning  LotFrontage SaleType SaleCondition  SalePrice
0   1          60       RL         65.0       WD        Normal     208500
1   2          20       RL         80.0       WD        Normal     181500
2   3          60       RL         68.0       WD        Normal     223500
3   4          70       RL         60.0       WD       Abnorml     140000
训练数据构型：
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-

## 1.2 数据预处理
从数据集中删除ID数据,将所有缺失的值替换为相应特征的平均值, 通过将特征重新缩放到零均值和单位方差来标准化数据。

In [21]:
# 数据预处理:从数据集中删除ID数据,将所有缺失的值替换为相应特征的平均值,
#           通过将特征重新缩放到零均值和单位方差来标准化数据。
all_features = pd.concat((train_data.iloc[:, 1:-1], test_data.iloc[:, 1:]))
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index
all_features[numeric_features] = all_features[numeric_features].apply(
    lambda x: (x - x.mean()) / (x.std()))
# 在标准化数据之后，将缺失值设置为0
all_features[numeric_features] = all_features[numeric_features].fillna(0)

# 类别特征转为数值标签，方法独热编码
# `Dummy_na=True` 将“na”（缺失值）视为有效的特征值，并为其创建指示符特征。
all_features = pd.get_dummies(all_features, dummy_na=True)
print('特征空间大小为：', all_features.shape)

特征空间大小为： (2919, 330)


## 2.1 构建神经网络

### 2.1.1 定义网络结构

In [22]:
class HimenoNet(nn.Module):
    def __init__(self, input_feature,
                 output_feature=1,
                 hidden_feature=512,
                 softmax_out=False):
        super(HimenoNet, self).__init__()

        self.mlp = nn.Sequential(
            nn.Linear(input_feature, hidden_feature),
            nn.ReLU(),
            nn.Linear(hidden_feature, hidden_feature),
        )
        
        if softmax_out:
            self.mlp.add_module('softmax', nn.Softmax(dim=1))
        else:
            self.mlp.add_module('relu', nn.ReLU())
        self.mlp.add_module('output', nn.Linear(hidden_feature, output_feature))

    def forward(self, x):
        x = self.mlp(x)
        return x

### 2.1.2 定义损失函数

In [ ]:
def himeno_loss(y_pred, y_true):
    return torch.mean((y_pred - y_true) ** 2)